In [ ]:
import os, time, datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import random
import logging
tqdm.pandas()
import seaborn as sns
from sklearn.model_selection import train_test_split

#NN Packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split,DataLoader, RandomSampler, SequentialSampler

logger = logging.getLogger(__name__)


if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

class SigirPreprocess():
    
    def __init__(self, text_data_path):
        self.text_data_path = text_data_path
        self.train = None
        self.dict_code_to_id = {}
        self.dict_id_to_code = {}
        self.list_tags = {}
        self.sentences = []
        self.labels = []
        self.text_col = None
        self.X_test = None
    def prepare_data(self ):
        catalog_eng= pd.read_csv(self.text_data_path+"data/catalog_english_taxonomy.tsv",sep="\t")
        X_train= pd.read_csv(self.text_data_path+"data/X_train.tsv",sep="\t")
        Y_train= pd.read_csv(self.text_data_path+"data/Y_train.tsv",sep="\t")
        
        self.list_tags = list(Y_train['Prdtypecode'].unique())
        for i,tag in enumerate(self.list_tags):
            self.dict_code_to_id[tag] = i 
            self.dict_id_to_code[i]=tag
        print(self.dict_code_to_id)
            
        Y_train['labels']=Y_train['Prdtypecode'].map(self.dict_code_to_id)
        train=pd.merge(left=X_train,right=Y_train,
               how='left',left_on=['Integer_id','Image_id','Product_id'],
               right_on=['Integer_id','Image_id','Product_id'])
        prod_map=pd.Series(catalog_eng['Top level category'].values,
                           index=catalog_eng['Prdtypecode']).to_dict()

        train['product'] = train['Prdtypecode'].map(prod_map)
        train['title_len']=train['Title'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['desc_len']=train['Description'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['title_desc_len']=train['title_len'] + train['desc_len']
        train.loc[train['Description'].isnull(), 'Description'] = " "
        train['title_desc'] = train['Title'] + " " + train['Description']
        
        self.train = train
        
    def get_sentences(self, text_col, remove_null_rows=False):
        self.text_col = text_col
        if remove_null_rows==True:
            new_train = self.train[self.train[text_col].notnull()]

        else:
            new_train = self.train.copy()
            
        self.sentences = new_train[text_col].values
        self.labels = new_train['labels'].values
    
    def prepare_test(self, text_col):
        X_test=pd.read_csv(self.text_data_path+"data/x_test_task1_phase1.tsv",sep="\t")
        X_test.loc[X_test['Description'].isnull(), 'Description'] = " "
        X_test['title_desc'] = X_test['Title'] + " " + X_test['Description']
        self.X_test = X_test
        self.test_sentences = X_test[text_col].values
        

In [ ]:
text_col = 'title_desc'
max_len = 256
val_size = 0.1

In [ ]:
Preprocess = SigirPreprocess("/kaggle/input/textphase1/")
Preprocess.prepare_data()
Preprocess.get_sentences(text_col, True)

In [ ]:
sentences = Preprocess.sentences
labels = Preprocess.labels
print("Total number of sentences:{}, labels:{}".format(len(sentences), len(labels)))

In [ ]:
#function to prepare input for model training
def prep_input(sentences,labels, max_len,tokenizer):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in tqdm(sentences):
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    if labels is not None:
        labels = torch.tensor(labels)
        return input_ids,attention_masks,labels
    else:
        return input_ids,attention_masks
    

In [ ]:
text_input='../input/multi-modal-input-text/'
tr_inputs_cam=torch.load(text_input+"tr_inputs_cam.pt")
val_inputs_cam=torch.load(text_input+"val_inputs_cam.pt")
tr_masks_cam=torch.load( text_input+"tr_masks_cam.pt")
val_masks_cam=torch.load( text_input+"val_masks_cam.pt")

tr_inputs_flau=torch.load(text_input+"tr_inputs_flau.pt")
val_inputs_flau=torch.load(text_input+"val_inputs_flau.pt")
tr_masks_flau=torch.load(text_input+"tr_masks_flau.pt")
val_masks_flau=torch.load(text_input+"val_masks_flau.pt")

In [ ]:
!pip install pretrainedmodels

In [ ]:
from transformers import CamembertConfig, CamembertTokenizer, CamembertModel, CamembertForSequenceClassification, AdamW
from transformers import FlaubertModel, FlaubertTokenizer,FlaubertForSequenceClassification,AdamW, FlaubertConfig 
from transformers.modeling_roberta import RobertaClassificationHead
from transformers.modeling_utils import SequenceSummary

In [ ]:
from torch.nn import functional as F
import torch.nn as nn
import pretrainedmodels
class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__["se_resnext50_32x4d"](pretrained=None)
        if pretrained is not None:
            self.base_model.load_state_dict(
                torch.load("../input/pretrained-model-weights-pytorch/se_resnext50_32x4d-a260b3a4.pth"
                )
            )
        self.l0 = nn.Linear(2048, 27)
    
    def forward(self, image):
        batch_size, _, _, _ = image.shape
        
        x = self.base_model.features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        
        out = self.l0(x)

        return out

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [ ]:
class vec_output_CamembertForSequenceClassification(CamembertModel):
    config_class = CamembertConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = CamembertModel(config)
        self.dense = nn.Linear(256*config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
#             output_attentions=output_attentions,
#             output_hidden_states=output_hidden_states,
        )
        sequence_output = outputs[0] #(B,256,768)
        x = sequence_output.view(sequence_output.shape[0], 256*768)
#         x = sequence_output[:, 0, :]  # take <s> token (equiv. to [CLS])-> #(B,768) Image -> (B,2048)
        x = self.dense(x)  # 768 -> 768
        feat= torch.tanh(x) 
        logits = self.out_proj(feat) # 768 -> 27
        outputs = (logits,) + outputs[2:]

        return outputs,feat  # (loss), logits, (hidden_states), (attentions)

In [ ]:
num_classes = 27

class vec_output_FlaubertForSequenceClassification(FlaubertModel):
    
    config_class = FlaubertConfig
    

    def __init__(self, config):
        super().__init__(config)
        self.transformer = FlaubertModel(config)
        self.sequence_summary = SequenceSummary(config)
        self.init_weights()
        self.dropout =  torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(config.hidden_size, num_classes)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        langs=None,
        token_type_ids=None,
        position_ids=None,
        lengths=None,
        cache=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        
        
        transformer_outputs = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            langs=langs,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            lengths=lengths,
            cache=cache,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        #output = self.dropout(output)
        output = transformer_outputs[0]
        vec = output[:,0]
        
        
        #logits
        dense = self.dropout(vec)
        
        #classifier
        logits = self.classifier(dense)
        
        outputs = (logits,) + transformer_outputs[1:]  # Keep new_mems and attention/hidden states if they are here
       
        
        return outputs,dense


### Image data prep

In [ ]:
catalog_eng= pd.read_csv("/kaggle/input/textphase1/data/catalog_english_taxonomy.tsv",sep="\t")
X_train= pd.read_csv("/kaggle/input/textphase1/data/X_train.tsv",sep="\t")
Y_train= pd.read_csv("/kaggle/input/textphase1/data/Y_train.tsv",sep="\t")
X_test=pd.read_csv("/kaggle/input/textphase1/data/x_test_task1_phase1.tsv",sep="\t")
dict_code_to_id = {}
dict_id_to_code={}
list_tags = list(Y_train['Prdtypecode'].unique())

for i,tag in enumerate(list_tags):
    dict_code_to_id[tag] = i 
    dict_id_to_code[i]=tag
Y_train['labels']=Y_train['Prdtypecode'].map(dict_code_to_id)
train=pd.merge(left=X_train,right=Y_train,
               how='left',left_on=['Integer_id','Image_id','Product_id'],
               right_on=['Integer_id','Image_id','Product_id'])
prod_map=pd.Series(catalog_eng['Top level category'].values,index=catalog_eng['Prdtypecode']).to_dict()
train['product']=train['Prdtypecode'].map(prod_map)

def get_img_path(img_id,prd_id,path):
    
    pattern = 'image'+'_'+str(img_id)+'_'+'product'+'_'+str(prd_id)+'.jpg'
    return path + pattern
train_img = train[['Image_id','Product_id','labels','product']]

train_img['image_path']=train_img.progress_apply(lambda x: get_img_path(x['Image_id'],x['Product_id'],
                                                                path = '/kaggle/input/imagetrain/image_training/'),axis=1)
X_test['image_path']=X_test.progress_apply(lambda x: get_img_path(x['Image_id'],x['Product_id'],
                                                    path='/kaggle/input/imagetest/image_test/image_test_task1_phase1/'),axis=1)
train_df, val_df, _, _ = train_test_split(train_img, train_img['labels'],random_state=2020, test_size = 0.1, stratify=train_img['labels'])

In [ ]:
input_size = 224 # for Resnt
# Applying Transforms to the Data
from torchvision import datasets, models, transforms

image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
from PIL import Image

class FusionDataset(Dataset):
    
    def __init__(self,df,inputs_cam,masks_cam,inputs_flau,masks_flau,transform=None,mode='train'):
        self.df = df
        self.transform=transform
        self.mode=mode
        self.inputs_cam=inputs_cam
        self.masks_cam=masks_cam
        self.inputs_flau=inputs_flau
        self.masks_flau=masks_flau
         
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        im_path = self.df.iloc[idx]['image_path']
        img = cv2.imread(im_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        img=img.cuda()
        input_id_cam=self.inputs_cam[idx].cuda()
        input_mask_cam=self.masks_cam[idx].cuda()
        input_id_flau=self.inputs_flau[idx].cuda()
        input_mask_flau=self.masks_flau[idx].cuda()
        
        if self.mode=='test':
            return img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau
        else:
#             labels = torch.tensor(self.df.iloc[idx]['labels'])
            labels = torch.tensor(self.df.iloc[idx]['labels']).cuda()             

            return img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau,labels

In [ ]:
a1 = torch.randn(3,10,10)


In [ ]:
reduce_dim=nn.Conv1d(in_channels = 10 , out_channels = 1 , kernel_size= 1)

In [ ]:
reduce_dim(a1).view(3,10).shape

In [ ]:
class vector_fusion(nn.Module):
    
    def __init__(self):
        super(vector_fusion, self).__init__()
        self.img_model = SEResnext50_32x4d(pretrained=None)
        self.img_model.load_state_dict(torch.load('../input/seresnext2048/best_model.pt'))
        self.img_model.l0=Identity()
        for params in self.img_model.parameters():
            params.requires_grad=False

        self.cam_model= vec_output_CamembertForSequenceClassification.from_pretrained(
    'camembert-base', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(Preprocess.dict_code_to_id), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False,) # Whether the model returns all hidden-states.
        
        
        cam_model_path = '../input/camembert-vec-256m768-10ep/best_model.pt'
        checkpoint = torch.load(cam_model_path)
        # model = checkpoint['model']
        self.cam_model.load_state_dict(checkpoint)
        for param in self.cam_model.parameters():
            param.requires_grad=False
        self.cam_model.out_proj=Identity()
        
        self.flau_model=vec_output_FlaubertForSequenceClassification.from_pretrained(
        'flaubert/flaubert_base_cased', 
        num_labels = len(Preprocess.dict_code_to_id), 
        output_attentions = False,
        output_hidden_states = False,)
        flau_model_path='../input/flaubert-8933/best_model.pt'
        checkpoint = torch.load(flau_model_path)
        self.flau_model.load_state_dict(checkpoint)
        for param in self.flau_model.parameters():
            param.requires_grad=False
        self.flau_model.classifier=Identity()
        

        self.reduce_dim=nn.Conv1d(in_channels = 2048 , out_channels = 768 , kernel_size= 1)
        self.reduce_dim2=nn.Conv1d(in_channels = 768 , out_channels = 1 , kernel_size= 1)
        self.out=nn.Linear(768*3, 27)
        
        #gamma
#         self.w1 = nn.Parameter(torch.zeros(1))
#         self.w2 = nn.Parameter(torch.zeros(1))
#         self.w3 = nn.Parameter(torch.zeros(1))
        
    def forward(self,img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau):
        
        cam_emb,vec1 =self.cam_model(input_id_cam, 
                     token_type_ids=None, 
                     attention_mask=input_mask_cam)
        flau_emb,vec2 =self.flau_model(input_id_flau, 
                     token_type_ids=None, 
                     attention_mask=input_mask_flau)
        
        #Projecting the image embedding to lower dimension
        img_emb=self.img_model(img)
        img_emb=img_emb.view(img_emb.shape[0],img_emb.shape[1],1)
        img_emb=self.reduce_dim(img_emb)
        img_emb=img_emb.view(img_emb.shape[0],img_emb.shape[1]) ###### bs * 768 
        
        #summing up the vectors
        #text_emb = cam_emb[0] + flau_emb[0]
        
        #Bilinear
        #text_emb = text_emb.view(text_emb.shape[0],1,text_emb.shape[1])  ##### bs * 1 * 768
        
        #Bilinear Pooling
        #pool_emb = torch.bmm(img_emb,text_emb) ### bs * 768 * 768
        #pool_emb = self.reduce_dim2(pool_emb).view(text_emb.shape[0],768)  #### bs * 1 * 768
        fuse= torch.cat([img_emb,cam_emb[0],flau_emb[0]],axis=1)
        
        logits=self.out(fuse)
        return logits
        
        
                  

In [ ]:
model=vector_fusion()

In [ ]:
model.cuda()

In [ ]:
train_dataset=FusionDataset(train_df,tr_inputs_cam,tr_masks_cam,tr_inputs_flau,tr_masks_flau,transform=image_transforms['test'])
val_dataset=FusionDataset(val_df,val_inputs_cam,val_masks_cam,val_inputs_flau,val_masks_flau,transform=image_transforms['test'])
# test_dataset=FusionDataset(X_test,test_inputs,test_makss,transform=image_transforms['test'],mode='test')

In [ ]:
batch_size=64
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
validation_dataloader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
# test_data=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import torch.nn as nn
loss_criterion = nn.CrossEntropyLoss()

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
from sklearn.metrics import f1_score

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    
    #tr and val
#     vec_output_tr = []
#     vec_output_val =[]

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    best_f1 = 0
    model.train()

    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader)):
        
        # Unpack this training batch from our dataloader. 
        #
        
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
#         return img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau

        b_img=batch[0].to(device)

        b_input_id_cam = batch[1].to(device)
        b_input_mask_cam = batch[2].to(device)
        b_input_id_flau = batch[3].to(device)
        b_input_mask_flau = batch[4].to(device)

        b_labels = batch[5].to(device)
        
        
        model.zero_grad()        

        
        logits = model(b_img,b_input_id_cam ,b_input_mask_cam,b_input_id_flau,b_input_mask_flau)
                            
        #Defining the loss
        loss = loss_criterion(logits, b_labels)
        
        #saving the features_tr
#         vec = vec.detach().cpu().numpy()
#         vec_output_tr.extend(vec)
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
        
        
        

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f} ".format(avg_train_loss))
    print("  Training epcoh took: {:} ".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    predictions=[]
    true_labels=[]
    

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_img=batch[0].to(device)

        b_input_id_cam = batch[1].to(device)
        b_input_mask_cam = batch[2].to(device)
        b_input_id_flau = batch[3].to(device)
        b_input_mask_flau = batch[4].to(device)

        b_labels = batch[5].to(device)
        
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():       
        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = model(b_img,b_input_id_cam ,b_input_mask_cam,b_input_id_flau,b_input_mask_flau)
            
        #new
        
        #defining the val loss
        loss = loss_criterion(logits, b_labels)
        
        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Move logits and labels to CPU
        predicted_labels=np.argmax(logits,axis=1)
        predictions.extend(predicted_labels)
        label_ids = b_labels.to('cpu').numpy()
        true_labels.extend(label_ids)
        
        #saving the features_tr
#         vec = vec.detach().cpu().numpy()
#         vec_output_val.extend(vec)
        

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    print("Validation F1-Score: {}".format(f1_score(true_labels,predictions,average='macro')))
    curr_f1=f1_score(true_labels,predictions,average='macro')
    if curr_f1 > best_f1:
        best_f1=curr_f1
        torch.save(model.state_dict(), 'best_model.pt')
#         np.save('best_vec_train_model_train.npy',vec_output_tr)
#         np.save('best_vec_val.npy',vec_output_val)
        
    # Record all statistics from this epoch.
#     training_stats.append(
#         {
#             'epoch': epoch_i + 1,
#             'Training Loss': avg_train_loss,
#             'Valid. Loss': avg_val_loss,
#             'Valid. Accur.': avg_val_accuracy,
#             'Training Time': training_time,
#             'Validation Time': validation_time
#         }
#     )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [49]:
from sklearn.metrics import f1_score
print("Validation F1-Score: {}".format(f1_score(true_labels,predictions,average='macro')))

Validation F1-Score: 0.9093939540881769
